In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/style2/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.04
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/to_star/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '__'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style2/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 500, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=100, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(m):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in m.named_parameters():
        sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        teacher_outputs = teacher_model(inputs)
        teacher_loss = criterion(teacher_outputs, targets)
        sp_alpha = 0
        sigmoid = nn.Sigmoid()
        sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if(epoch+1)%100 == 0:
        test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


        logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
        is_best = test_auroc+source_auroc > best_acc
        best_acc = max(test_auroc+source_auroc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()

#     if (epoch+1)%4000 == 0:
#         teacher_model.load_state_dict(student_model.state_dict())
#     teacher_model.eval()


Epoch: [1 | 5000] LR: 0.040000
Train | 16/16 | Loss:1.0295 | MainLoss:0.9157 | Alpha:0.0319 | SPLoss:3.7503 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [2 | 5000] LR: 0.041200
Train | 16/16 | Loss:1.0033 | MainLoss:0.6928 | Alpha:0.0318 | SPLoss:9.7631 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [3 | 5000] LR: 0.042400
Train | 16/16 | Loss:0.9470 | MainLoss:0.6944 | Alpha:0.0300 | SPLoss:8.4083 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [4 | 5000] LR: 0.043600
Train | 16/16 | Loss:0.8587 | MainLoss:0.6937 | Alpha:0.0289 | SPLoss:5.7314 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [5 | 5000] LR: 0.044800
Train | 16/16 | Loss:0.8095 | MainLoss:0.6954 | Alpha:0.0315 | SPLoss:3.6073 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [6 | 5000] LR: 0.046000
Train | 16/16 | Loss:0.7634 | MainLoss:0.6944 | Alpha:0.0317 | SPLoss:2.1725 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [7 | 5000] LR: 0.047200
Train | 16/16 | Loss:0.7332 | MainLoss:0.6944 | Alpha:0.0307 | SPLoss:1.2740 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [8 | 

Train | 16/16 | Loss:0.7458 | MainLoss:0.6897 | Alpha:0.0311 | SPLoss:1.8126 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [60 | 5000] LR: 0.110800
Train | 16/16 | Loss:0.7193 | MainLoss:0.6645 | Alpha:0.0319 | SPLoss:1.7151 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [61 | 5000] LR: 0.112000
Train | 16/16 | Loss:0.7294 | MainLoss:0.6668 | Alpha:0.0317 | SPLoss:1.9200 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [62 | 5000] LR: 0.113200
Train | 16/16 | Loss:0.7320 | MainLoss:0.6566 | Alpha:0.0302 | SPLoss:2.4807 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [63 | 5000] LR: 0.114400
Train | 16/16 | Loss:0.7241 | MainLoss:0.6577 | Alpha:0.0318 | SPLoss:2.0895 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [64 | 5000] LR: 0.115600
Train | 16/16 | Loss:0.7346 | MainLoss:0.6732 | Alpha:0.0309 | SPLoss:1.9857 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [65 | 5000] LR: 0.116800
Train | 16/16 | Loss:0.7131 | MainLoss:0.6601 | Alpha:0.0321 | SPLoss:1.6434 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [66 | 5000] LR: 0.118000
Train 

Train | 16/16 | Loss:2.6551 | MainLoss:0.6831 | Alpha:0.0326 | SPLoss:61.2011 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [116 | 5000] LR: 0.159997
Train | 16/16 | Loss:2.2505 | MainLoss:0.6889 | Alpha:0.0313 | SPLoss:49.9951 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [117 | 5000] LR: 0.159996
Train | 16/16 | Loss:2.4979 | MainLoss:0.6848 | Alpha:0.0310 | SPLoss:57.8768 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [118 | 5000] LR: 0.159996
Train | 16/16 | Loss:3.2227 | MainLoss:0.9120 | Alpha:0.0300 | SPLoss:81.3432 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [119 | 5000] LR: 0.159995
Train | 16/16 | Loss:9.1708 | MainLoss:0.6948 | Alpha:0.0307 | SPLoss:274.6574 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [120 | 5000] LR: 0.159995
Train | 16/16 | Loss:4.8228 | MainLoss:0.6991 | Alpha:0.0316 | SPLoss:127.6301 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [121 | 5000] LR: 0.159994
Train | 16/16 | Loss:1.3757 | MainLoss:0.6990 | Alpha:0.0331 | SPLoss:21.4789 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [122 | 5000] LR:

Train | 16/16 | Loss:119459.9269 | MainLoss:0.7860 | Alpha:0.0287 | SPLoss:4072575.7500 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [173 | 5000] LR: 0.159920
Train | 16/16 | Loss:138559.3936 | MainLoss:0.7043 | Alpha:0.0305 | SPLoss:4524346.0000 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [174 | 5000] LR: 0.159918
Train | 16/16 | Loss:76246.9788 | MainLoss:0.7038 | Alpha:0.0310 | SPLoss:2458637.7500 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [175 | 5000] LR: 0.159916
Train | 16/16 | Loss:60412.3921 | MainLoss:0.7009 | Alpha:0.0316 | SPLoss:1940121.6250 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [176 | 5000] LR: 0.159914
Train | 16/16 | Loss:12868.1950 | MainLoss:0.7009 | Alpha:0.0294 | SPLoss:436519.6562 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [177 | 5000] LR: 0.159911
Train | 16/16 | Loss:1092.2496 | MainLoss:0.6958 | Alpha:0.0318 | SPLoss:34047.6289 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [178 | 5000] LR: 0.159909
Train | 16/16 | Loss:130.7355 | MainLoss:0.6946 | Alpha:0.0319 | SPLoss:4120.5903 | 

Train | 16/16 | Loss:44.4559 | MainLoss:0.6994 | Alpha:0.0324 | SPLoss:1301.5159 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [228 | 5000] LR: 0.159749
Train | 16/16 | Loss:6.8972 | MainLoss:0.7062 | Alpha:0.0312 | SPLoss:190.9130 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [229 | 5000] LR: 0.159745
Train | 16/16 | Loss:183.4992 | MainLoss:0.6969 | Alpha:0.0306 | SPLoss:6114.9863 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [230 | 5000] LR: 0.159741
Train | 16/16 | Loss:176.6848 | MainLoss:0.6959 | Alpha:0.0316 | SPLoss:5464.2212 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [231 | 5000] LR: 0.159737
Train | 16/16 | Loss:34.2445 | MainLoss:0.6886 | Alpha:0.0309 | SPLoss:1048.6112 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [232 | 5000] LR: 0.159733
Train | 16/16 | Loss:4.4017 | MainLoss:0.6853 | Alpha:0.0314 | SPLoss:116.7327 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [233 | 5000] LR: 0.159729
Train | 16/16 | Loss:203469.6882 | MainLoss:0.9624 | Alpha:0.0305 | SPLoss:7155145.0000 | CLSLoss:0.0000 | AUROC:0.0000


...

Train | 16/16 | Loss:0.4990 | MainLoss:0.2952 | Alpha:0.0312 | SPLoss:6.5353 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1575 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.4991 | MainLoss:0.3000 | Alpha:0.0305 | SPLoss:6.5310 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1576 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.5359 | MainLoss:0.3226 | Alpha:0.0327 | SPLoss:6.5282 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1577 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.4922 | MainLoss:0.2906 | Alpha:0.0309 | SPLoss:6.5252 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1578 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.4991 | MainLoss:0.2941 | Alpha:0.0314 | SPLoss:6.5219 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1579 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.5058 | MainLoss:0.3034 | Alpha:0.0310 | SPLoss:6.5187 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1580 | 5000] LR: 0.000128
Train | 16/16 | Loss:0.5129 | MainLoss:0.3138 | Alpha:0.0305 | SPLoss:6.5168 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1581 | 5000] LR: 0

Train | 16/16 | Loss:0.4710 | MainLoss:0.2774 | Alpha:0.0304 | SPLoss:6.3765 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1631 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.4765 | MainLoss:0.2878 | Alpha:0.0296 | SPLoss:6.3743 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1632 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.4958 | MainLoss:0.2934 | Alpha:0.0318 | SPLoss:6.3719 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1633 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.4943 | MainLoss:0.2951 | Alpha:0.0313 | SPLoss:6.3672 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1634 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.5034 | MainLoss:0.3062 | Alpha:0.0310 | SPLoss:6.3636 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1635 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.5159 | MainLoss:0.3206 | Alpha:0.0307 | SPLoss:6.3610 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1636 | 5000] LR: 0.000126
Train | 16/16 | Loss:0.4789 | MainLoss:0.2794 | Alpha:0.0314 | SPLoss:6.3582 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1637 | 5000] LR: 0

Train | 16/16 | Loss:0.4886 | MainLoss:0.2869 | Alpha:0.0324 | SPLoss:6.2179 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1688 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4636 | MainLoss:0.2742 | Alpha:0.0305 | SPLoss:6.2151 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1689 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4952 | MainLoss:0.3001 | Alpha:0.0314 | SPLoss:6.2130 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1690 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4952 | MainLoss:0.2919 | Alpha:0.0327 | SPLoss:6.2101 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1691 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4877 | MainLoss:0.2820 | Alpha:0.0331 | SPLoss:6.2070 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1692 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4765 | MainLoss:0.2918 | Alpha:0.0298 | SPLoss:6.2042 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1693 | 5000] LR: 0.000123
Train | 16/16 | Loss:0.4933 | MainLoss:0.3060 | Alpha:0.0302 | SPLoss:6.2027 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1694 | 5000] LR: 0

Train | 16/16 | Loss:0.5209 | MainLoss:0.3209 | Alpha:0.0329 | SPLoss:6.0809 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1744 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.4762 | MainLoss:0.2929 | Alpha:0.0302 | SPLoss:6.0769 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1745 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.4878 | MainLoss:0.2974 | Alpha:0.0313 | SPLoss:6.0752 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1746 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.5147 | MainLoss:0.3210 | Alpha:0.0319 | SPLoss:6.0735 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1747 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.4960 | MainLoss:0.3023 | Alpha:0.0319 | SPLoss:6.0705 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1748 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.4805 | MainLoss:0.2890 | Alpha:0.0316 | SPLoss:6.0682 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1749 | 5000] LR: 0.000121
Train | 16/16 | Loss:0.4745 | MainLoss:0.2770 | Alpha:0.0326 | SPLoss:6.0655 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1750 | 5000] LR: 0

Train | 16/16 | Loss:0.4817 | MainLoss:0.3013 | Alpha:0.0303 | SPLoss:5.9543 | CLSLoss:0.0000 | AUROC:0.0000
Test | 122/16 | Loss:0.4578 | MainLoss:0.4578 | SPLoss:5.9533 | CLSLoss:0.0000 | AUROC:0.9072
Test | 31/16 | Loss:0.2281 | MainLoss:0.2281 | SPLoss:5.9533 | CLSLoss:0.0000 | AUROC:0.9792

Epoch: [1801 | 5000] LR: 0.000119
Train | 16/16 | Loss:0.4647 | MainLoss:0.2789 | Alpha:0.0312 | SPLoss:5.9521 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1802 | 5000] LR: 0.000119
Train | 16/16 | Loss:0.4976 | MainLoss:0.3088 | Alpha:0.0317 | SPLoss:5.9495 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1803 | 5000] LR: 0.000118
Train | 16/16 | Loss:0.4871 | MainLoss:0.3010 | Alpha:0.0313 | SPLoss:5.9470 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1804 | 5000] LR: 0.000118
Train | 16/16 | Loss:0.4988 | MainLoss:0.3113 | Alpha:0.0315 | SPLoss:5.9445 | CLSLoss:0.0000 | AUROC:0.0000

Epoch: [1805 | 5000] LR: 0.000118
Train | 16/16 | Loss:0.4628 | MainLoss:0.2757 | Alpha:0.0315 | SPLoss:5.9424 | CLSLoss:0.0000